In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_blobs, load_iris, load_wine
from sklearn.metrics import mean_squared_error
from sklearn.cluster import KMeans

# Yeni optimizasyon algoritması (örnek: Karınca Kolonisi + Lojistik Harita)
def yeni_optimizasyon_algoritmasi(veri, kume_sayisi, iterasyon_sayisi=30):
    np.random.seed(42)
    en_iyi_sonuç = float('inf')
    for _ in range(iterasyon_sayisi):
        lojistik_harita = np.random.rand(len(veri)) * 0.9 + 0.1
        yeni_veri = veri * lojistik_harita[:, np.newaxis]
        kmeans = KMeans(n_clusters=kume_sayisi, random_state=42).fit(yeni_veri)
        hata = mean_squared_error(veri, kmeans.cluster_centers_[kmeans.labels_])
        if hata < en_iyi_sonuç:
            en_iyi_sonuç = hata
    return en_iyi_sonuç

# Veri setleri oluşturuluyor
def verisetlerini_olustur():
    veri_dict = {}
    veri_dict['Aggregation'] = make_blobs(n_samples=500, centers=5, cluster_std=0.60, random_state=42)[0]
    veri_dict['Iris'] = load_iris().data
    veri_dict['Wine'] = load_wine().data
    return veri_dict

# Sonuçları hesaplama
def sonuçları_hesapla():
    verisetleri = verisetlerini_olustur()
    sonuçlar = []
    for veri_ismi, veri in verisetleri.items():
        sonuç = yeni_optimizasyon_algoritmasi(veri, kume_sayisi=3)
        sonuçlar.append({'Veri Seti': veri_ismi, 'Sonuç': sonuç})
    return pd.DataFrame(sonuçlar)

# Sonuçları ekrana yazdır
sonuç_df = sonuçları_hesapla()
print(sonuç_df)


     Veri Seti         Sonuç
0  Aggregation     12.201756
1         Iris      3.874264
2         Wine  12897.929281


In [2]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_blobs, load_iris, load_wine
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error

# Kaotik harita fonksiyonu
def kaotik_harita(boyut, iterasyon):
    kaotik_dizi = np.zeros((iterasyon, boyut))
    kaotik_dizi[0, :] = np.random.rand(boyut)
    for i in range(1, iterasyon):
        kaotik_dizi[i, :] = 4 * kaotik_dizi[i - 1, :] * (1 - kaotik_dizi[i - 1, :])
    return kaotik_dizi

# Kaotik Fox algoritması
def kaotik_fox_algoritmasi(veri, kume_sayisi, iterasyon_sayisi=30):
    en_iyi_sonuç = float('inf')
    tüm_sonuçlar = []
    for _ in range(iterasyon_sayisi):
        kaotik_dizi = kaotik_harita(len(veri), 1)[0]
        yeni_veri = veri * kaotik_dizi[:, np.newaxis]
        kmeans = KMeans(n_clusters=kume_sayisi, random_state=42).fit(yeni_veri)
        hata = mean_squared_error(veri, kmeans.cluster_centers_[kmeans.labels_])
        tüm_sonuçlar.append(hata)
        if hata < en_iyi_sonuç:
            en_iyi_sonuç = hata
    return en_iyi_sonuç, tüm_sonuçlar

# Veri setleri oluşturuluyor
def verisetlerini_olustur():
    veri_dict = {}
    veri_dict['Aggregation'] = make_blobs(n_samples=500, centers=5, cluster_std=0.60, random_state=42)[0]
    veri_dict['Iris'] = load_iris().data
    veri_dict['Wine'] = load_wine().data
    return veri_dict

# Sonuçları hesaplama ve Excel'e kaydetme
def sonuçları_hesapla_ve_kaydet():
    verisetleri = verisetlerini_olustur()
    sonuç_listesi = []
    for veri_ismi, veri in verisetleri.items():
        en_iyi_sonuç, tüm_sonuçlar = kaotik_fox_algoritmasi(veri, kume_sayisi=3)
        sonuç_listesi.append({
            'Veri Seti': veri_ismi,
            'Minimum': np.min(tüm_sonuçlar),
            'Maksimum': np.max(tüm_sonuçlar),
            'Ortalama': np.mean(tüm_sonuçlar),
            'Standart Sapma': np.std(tüm_sonuçlar)
        })
    sonuç_df = pd.DataFrame(sonuç_listesi)
    sonuç_df.to_excel("sonuçlar.xlsx", index=False)
    print("Sonuçlar 'sonuçlar.xlsx' dosyasına kaydedildi.")

# Fonksiyonu çalıştır
sonuçları_hesapla_ve_kaydet()


Sonuçlar 'sonuçlar.xlsx' dosyasına kaydedildi.


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs, load_iris, load_wine
from sklearn.metrics import pairwise_distances_argmin_min

# Logistic kaotik harita
def logistic_kaotik_harita(boyut, iterasyon):
    kaotik_dizi = np.zeros((iterasyon, boyut))
    kaotik_dizi[0, :] = np.random.rand(boyut)
    for i in range(1, iterasyon):
        kaotik_dizi[i, :] = 4 * kaotik_dizi[i - 1, :] * (1 - kaotik_dizi[i - 1, :])
    return kaotik_dizi

# Karınca Kolonisi Optimizasyonu ile kümeleme
def karinca_kolonisi_kaotik(veri, kume_sayisi, iterasyon_sayisi=30, karinca_sayisi=20, alfa=1, beta=2, buharlasma=0.5):
    n, m = veri.shape
    feromon = logistic_kaotik_harita(kume_sayisi, n)
    en_iyi_sonuç = float('inf')
    en_iyi_merkezler = None
    tüm_sonuçlar = []

    for iterasyon in range(iterasyon_sayisi):
        # Karıncaların hareketi
        merkezler = np.zeros((karinca_sayisi, kume_sayisi, m))
        for k in range(karinca_sayisi):
            for c in range(kume_sayisi):
                merkezler[k, c, :] = veri[np.random.choice(n)]

        # Kümeleme işlemi
        for k in range(karinca_sayisi):
            etiketler, en_yakin_merkez = pairwise_distances_argmin_min(veri, merkezler[k])
            toplam_hata = np.sum(en_yakin_merkez**2)
            tüm_sonuçlar.append(toplam_hata)
            if toplam_hata < en_iyi_sonuç:
                en_iyi_sonuç = toplam_hata
                en_iyi_merkezler = merkezler[k]

        # Feromon güncellemesi
        feromon = (1 - buharlasma) * feromon + np.random.rand(*feromon.shape) * toplam_hata

    return en_iyi_sonuç, tüm_sonuçlar

# Veri setleri oluşturuluyor
def verisetlerini_olustur():
    veri_dict = {}
    veri_dict['Aggregation'] = make_blobs(n_samples=500, centers=5, cluster_std=0.60, random_state=42)[0]
    veri_dict['Iris'] = load_iris().data
    veri_dict['Wine'] = load_wine().data
    return veri_dict

# Sonuçları hesaplama ve gösterme
def sonuçları_hesapla_ve_göster():
    verisetleri = verisetlerini_olustur()
    sonuç_listesi = []
    for veri_ismi, veri in verisetleri.items():
        en_iyi_sonuç, tüm_sonuçlar = karinca_kolonisi_kaotik(veri, kume_sayisi=3)
        sonuç_listesi.append({
            'Veri Seti': veri_ismi,
            'Minimum': np.min(tüm_sonuçlar),
            'Maksimum': np.max(tüm_sonuçlar),
            'Ortalama': np.mean(tüm_sonuçlar),
            'Standart Sapma': np.std(tüm_sonuçlar)
        })

    sonuç_df = pd.DataFrame(sonuç_listesi)

    # Tabloyu göster
    display(sonuç_df)



# Fonksiyonu çalıştır
sonuçları_hesapla_ve_göster()


,Veri Seti,Minimum,Maksimum,Ortalama,Standart Sapma
0,Aggregation,4.567243e+03,8.965734e+04,2.367664e+04,1.218346e+04
1,Iris,9.407000e+01,1.947950e+03,3.978608e+02,3.315505e+02
2,Wine,2.473411e+06,4.143090e+07,9.770757e+06,6.953778e+06
